<a href="https://colab.research.google.com/github/yootazi/Training-_a_NN_on_MNIST_dataset/blob/main/Training_a_NN_on_MNIST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**I- Downloading the MNIST Dataset**

**II- Creating a Data Loader**

**III- Building a Model**

**IV- Training the Model**

**V- Saving the Trained Model**

In [ ]:
#@title **Connecting to Google Drive** { form-width: "50%" }
from google.colab import drive
drive.mount('/content/gdrive/')


In [ ]:
#@title **Importing Libraries** { form-width: "50%" }
!pip install torchaudio librosa boto3

import torchvision   # torchvision is a package in the PyTorch library containing computer-vision models, datasets, and image transformations
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets    
from torchvision.transforms import ToTensor


In [ ]:
#@title **Creating and Training the Model with the following parameters:** { form-width: "50%" }


LEARNING_RATE = 0.001 #@param {type:"raw"}
EPOCHS =  10 #@param {type:"integer"}
BATCH_SIZE = 128 #@param {type:"integer"}


class FeedForwardNet(nn.Module):                                         # Building the model - inherits from imported nn.Module

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()           # storing input layer of the model in this class
        self.dense_layers = nn.Sequential(    # storing hidden and output layers packed together (nn.Sequential)
            nn.Linear(28 * 28, 256),          # dense layer (input features(MNIST measures), output features)
            nn.ReLU(),                        # applying activation function
            nn.Linear(256, 10)                # output layer(number of input, number of output)
        )
        self.softmax = nn.Softmax(dim=1)      # nn.Softmax transforms values in a way that the sum == 1 (normalisation)

    def forward(self, input_data):            # defining the forward method to tell pytorch in what sequence it should manipilate the data
        x = self.flatten(input_data)
        logits = self.dense_layers(x)         # passing input data to dense layer
        predictions = self.softmax(logits)    # normalising the output data
        return predictions


def download_mnist_datasets():                                           # Load MNIST Dataset (70,000 handwritten numeric digit images and their respective labels- 60000 train_set, 10000 test_set) from PyTorch Torchvision
   
    train_data = datasets.MNIST(                        # train set
        root="/content/gdrive/MyDrive/musicdata/",      # where to store the dataset
        train=True,                                     # we want the train set of the dataset
        download=True,
        transform=ToTensor(),                           # ToTensor reshapes a new tensor where each value is normalised between 0 and 1
    )
    validation_data = datasets.MNIST(                   # test set
        root="/content/gdrive/MyDrive/musicdata/",
        train=False,                                    # we want the non-trained part of the dataset (won't be used in training our model)
        download=True,
        transform=ToTensor(),
    )
    return train_data, validation_data
  

def create_data_loader(train_data, batch_size):                          # DataLoader fetches data in batches - combines the dataset and a sampler, returning an iterable over the dataset
    
    train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE)    # BATCH_SIZE is defined in the previous cell
    return train_dataloader


def train_single_epoch(model, data_loader, loss_fn, optimiser, device):    
    for input, target in data_loader:                                    # looping through all the samples in the dataset getting a new batch of samples in each iteration returning x == input and y == target for one batch
        input, target = input.to(device), target.to(device)

        prediction = model(input)                                        # calculating loss
        loss = loss_fn(prediction, target)                               # loos_fn will be defined later

        optimiser.zero_grad()                                            # backpropagate error and update weights (reseting grediant to zero for each training iteration == zero_grad)
        loss.backward()
        optimiser.step()

    print(f"loss: {loss.item()}")


def train(model, data_loader, loss_fn, optimiser, device, epochs):    # will go through all epochs
    for i in range(epochs):
        print(f"Epoch {i+1}")
        train_single_epoch(model, data_loader, loss_fn, optimiser, device)
        print("---------------------------")
    print("Finished training")


if __name__ == "__main__":

    # download data and create data loader
    train_data, _ = download_mnist_datasets()     # not using validation_data (_)
    print("MNIST dataset downloaded")

    train_dataloader = create_data_loader(train_data, BATCH_SIZE)
    

    # construct model and assign it to device
    if torch.cuda.is_available():    # checking if gpu acceleration is available
        device = "cuda"
    else:
        device = "cpu"
    print(f"Using {device}")
    feed_forward_net = FeedForwardNet().to(device)
    print(feed_forward_net)

    # initialise loss funtion + optimiser
    loss_fn = nn.CrossEntropyLoss()
    optimiser = torch.optim.Adam(feed_forward_net.parameters(),
                                 lr=LEARNING_RATE)

    # train model
    train(feed_forward_net, train_dataloader, loss_fn, optimiser, device, EPOCHS)

    # save model
    path = '/content/gdrive/MyDrive/ai_music_projects/training_NN_on_MNIST_dataset'
    torch.save(feed_forward_net.state_dict(), "/content/gdrive/MyDrive/ai_music_projects/training_NN_on_MNIST_dataset/feedforward.pth")
    print("Trained feed forward net saved in {}".format(path))
